## Production Planning Project: **Phase 2**


### Details

Full Name: **Hamed Araab**

Student Number: **9925003**


### Dependencies

First, we import the libraries that we are going to need later on.

- `pandas` and `numpy` for minor data manipulations.
- `pulp` for creating and solving the ILP model.

In [255]:
import pandas as pd
import numpy as np

from pulp import LpProblem, LpMinimize, LpVariable, LpInteger, lpSum

### Dataset

Here, we import the forecasting data from the previous phase.

In [256]:
phase_1_output = pd.read_excel("../phase_1/output.xlsx", sheet_name="Data", index_col=0)

phase_1_output

,G1 Actual,G2 Actual,G3 Actual,G1 Forecast (SES),G2 Forecast (SES),G3 Forecast (SES),G1 Forecast (SMA),G2 Forecast (SMA),G3 Forecast (SMA),G1 Forecast (WMA),G2 Forecast (WMA),G3 Forecast (WMA),G1 Forecast (LR),G2 Forecast (LR),G3 Forecast (LR),G1 Forecast (ALR),G2 Forecast (ALR),G3 Forecast (ALR)
0,352.1,146.8,14.7,352.100000,146.800000,14.700000,352.100000,146.800000,14.700000,352.10000,146.800000,14.700000,395.830000,182.258571,17.584286,334.085280,135.515172,14.582346
1,469.3,171.7,16.2,352.100000,146.800000,14.700000,469.300000,171.700000,16.200000,469.30000,171.700000,16.200000,395.457368,182.059248,17.071203,426.822969,160.227161,17.641170
2,498.0,229.3,22.7,387.260000,154.270000,15.150000,498.000000,229.300000,22.700000,498.00000,229.300000,22.700000,395.084737,181.859925,16.558120,508.436307,230.524012,21.458910
3,476.5,220.7,19.2,420.482000,176.779000,17.415000,439.800000,182.600000,17.866667,460.21000,195.520000,19.150000,394.712105,181.660602,16.045038,487.321661,222.172337,18.686954
4,434.7,206.6,17.7,437.287400,189.955300,17.950500,481.266667,207.233333,19.366667,481.51000,213.480000,19.650000,394.339474,181.461278,15.531955,454.033856,207.979807,17.713604
5,405.4,185.6,14.7,436.511180,194.948710,17.875350,469.733333,218.866667,19.866667,459.90000,215.370000,19.150000,393.966842,181.261955,15.018872,413.492525,186.840937,15.739125
6,388.2,176.7,10.2,427.177826,192.144097,16.922745,438.866667,204.300000,17.200000,428.41000,198.920000,16.500000,393.594211,181.062632,14.505789,395.386796,177.730740,13.005491
7,311.8,176.4,14.7,415.484478,187.510868,14.905921,409.433333,189.633333,14.200000,402.66000,185.350000,13.050000,393.221579,180.863308,13.992707,349.805020,175.707265,13.510873
8,301.4,168.1,12.2,384.379135,184.177608,14.844145,368.466667,179.566667,13.200000,353.44000,178.330000,13.350000,392.848947,180.663985,13.479624,301.400000,168.100000,12.200000
9,291.4,161.1,10.5,359.485394,179.354325,14.050902,333.800000,173.733333,12.366667,321.88000,172.310000,12.550000,392.476316,180.464662,12.966541,291.400000,161.100000,10.500000


### Presets

The presets of the problem are the following:

In [ ]:
selector = [
    "G1 Forecast (WMA)",
    "G2 Forecast (ALR)",
    "G3 Forecast (ALR)",
]                                                   # the optimal forecasting methods according
                                                    # to the results from the last phase

T = range(20, 26)                                   # periods
G = range(1, 4)                                     # groups

D = (
    phase_1_output[selector]
        .rename(columns=dict(zip(selector, G)))
        .transpose()
        .pipe(np.ceil)[T]
)                                                   # demands

### Variables

The variables of the problem are the following:

In [ ]:
P = LpVariable.dicts('P', (T, G), lowBound=0, cat=LpInteger)    # regular production (units)
O = LpVariable.dicts('O', (T, G), lowBound=0, cat=LpInteger)    # overtime production (units)
Λ = LpVariable.dicts('Λ', (T, G), lowBound=0, cat=LpInteger)    # production increase (units)
Ω = LpVariable.dicts('Ω', (T, G), lowBound=0, cat=LpInteger)    # production decrease (units)
I = LpVariable.dicts('I', (T, G), lowBound=0, cat=LpInteger)    # net stored products (units)
L = LpVariable.dicts('L', (T, G), lowBound=0, cat=LpInteger)    # storage level (units)
E = LpVariable.dicts('E', (T, G), lowBound=0, cat=LpInteger)    # shortage (units)
W = LpVariable.dicts('W', T, lowBound=0, cat=LpInteger)         # workers (worker)
H = LpVariable.dicts('H', T, lowBound=0, cat=LpInteger)         # workers' increase (worker)
F = LpVariable.dicts('F', T, lowBound=0, cat=LpInteger)         # workers' decrease (worker)

### ILP Model

Here, we define our ILP model using `pulp`. To do so, we define a function that returns the model, passing the model's parameters to the function in order to perform sensitivity analysis with ease.

In [257]:
def get_model(
    a = 2.5,                                                            # regular production rate (units per worker)
    b = 0.625,                                                          # overtime production rate (units per worker)
    p = dict(zip(G, [11_300_000, 12_200_000, 16_700_000])),             # regular production cost (tomans per unit)
    o = dict(zip(G, [13_560_000, 14_640_000, 20_040_000])),             # overtime production cost (tomans per unit)
    λ = 1_000_000,                                                      # production increase cost (tomans per unit)
    ω = 1_500_000,                                                      # production decrease cost (tomans per unit)
    l = dict(zip(G, [2_300_000 / 12, 3_100_000 / 12, 5_600_000 / 12])), # storage cost (tomans per unit)
    e = 0,                                                              # shortage cost (tomans per unit)
    w = 15_000_000,                                                     # regular salary (tomans per worker)
    h = 2_400_000,                                                      # hiring cost (tomans per worker)
    f = 12_000_000,                                                     # firing cost (tomans per worker)    
):
    model = LpProblem(name="production_planning", sense=LpMinimize)

    model += (
        lpSum([p[g] * P[t][g] + o[g] * O[t][g] + λ * Λ[t][g] + ω * Ω[t][g] + l[g] * L[t][g] + e * E[t][g] for g in G for t in T]) +
        lpSum([w * W[t] + h * H[t] + f * F[t] for t in T])
    )

    for t in T:
        model += lpSum([P[t][g] for g in G]) <= a * W[t]
        model += lpSum([O[t][g] for g in G]) <= b * W[t]
        
        if t > 20:
            model += W[t] == W[t - 1] + H[t] - F[t]
        else:
            model += W[t] == 20_000 + H[t] - F[t]

        for g in G:
            model += I[t][g] == L[t][g] - E[t][g]

            if t > 20:
                model += P[t][g] == P[t - 1][g] + Λ[t][g] - Ω[t][g]
                model += I[t][g] == I[t - 1][g] + P[t][g] + O[t][g] - D[t][g]  
            else:
                model += P[t][g] == Λ[t][g] - Ω[t][g]
                model += I[t][g] == P[t][g] + O[t][g] - D[t][g]
            
            if t == 25:
                model += I[t][g] == 0

    return model

Now, we can get our model by calling `get_model`:

In [258]:
model = get_model()

Subsequently, we call `model.solve` to solve it using PuLP:

In [259]:
model.solve()

1

### Results

We can depict the model's results using `pd.DataFrame`:

In [260]:
print("total cost:", model.objective.value())

pd.DataFrame({
    "P": {t: {g: P[t][g].value() for g in G} for t in T},
    "O": {t: {g: O[t][g].value() for g in G} for t in T},
    "Λ": {t: {g: Λ[t][g].value() for g in G} for t in T},
    "Ω": {t: {g: Ω[t][g].value() for g in G} for t in T},
    "I": {t: {g: I[t][g].value() for g in G} for t in T},
    "L": {t: {g: L[t][g].value() for g in G} for t in T},
    "E": {t: {g: E[t][g].value() for g in G} for t in T},
    "W": {t: W[t].value() for t in T},
    "H": {t: H[t].value() for t in T},
    "F": {t: F[t].value() for t in T},
}).transpose()

total cost: 298286495000.0


,20,21,22,23,24,25
P,"{1: 321.0, 2: 163.0, 3: 7.0}","{1: 321.0, 2: 153.0, 3: 6.0}","{1: 321.0, 2: 148.0, 3: 6.0}","{1: 321.0, 2: 144.0, 3: 5.0}","{1: 321.0, 2: 144.0, 3: 5.0}","{1: 321.0, 2: 144.0, 3: 5.0}"
O,"{1: 120.0, 2: 3.0, 3: 0.0}","{1: 114.0, 2: 6.0, 3: 0.0}","{1: 112.0, 2: 6.0, 3: 0.0}","{1: 113.0, 2: 4.0, 3: 0.0}","{1: 114.0, 2: 0.0, 3: 0.0}","{1: 114.0, 2: 1.0, 3: 0.0}"
Λ,"{1: 321.0, 2: 163.0, 3: 7.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}"
Ω,"{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 10.0, 3: 1.0}","{1: 0.0, 2: 5.0, 3: 0.0}","{1: 0.0, 2: 4.0, 3: 1.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}"
I,"{1: 1.0, 2: 0.0, 3: 0.0}","{1: 1.0, 2: 0.0, 3: 0.0}","{1: 1.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 12.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}"
L,"{1: 1.0, 2: 0.0, 3: 0.0}","{1: 1.0, 2: 0.0, 3: 0.0}","{1: 1.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 12.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}"
E,"{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}","{1: 0.0, 2: 0.0, 3: 0.0}"
W,197.0,192.0,190.0,188.0,188.0,188.0
H,0.0,0.0,0.0,0.0,0.0,0.0
F,19803.0,5.0,2.0,2.0,0.0,0.0


Since salaries are much higher than firing cost and we have too many workers, we need to fire the majority of workers and adopt a strategy similar to Level.

### Sensitivity Analysis

In the final section, we are going to perform sensitivity analysis on `w` and `f`.

#### `w`: Salary (Tomans per Worker)

In [261]:
for w in [12e6, 13e6, 14e6, 15e6, 16e6, 17e6, 18e6]:
    model = get_model(w=w)
    
    print(f"w = {int(w)}, total cost:", model.objective.value())

w = 12000000, total cost: 294857495000.0
w = 13000000, total cost: 296000495000.0
w = 14000000, total cost: 297143495000.0
w = 15000000, total cost: 298286495000.0
w = 16000000, total cost: 299429495000.0
w = 17000000, total cost: 300572495000.0
w = 18000000, total cost: 301715495000.0


#### `f`: Fire Cost (Tomans per Worker)

In [262]:
for f in [6e6, 8e6, 10e6, 12e6, 14e6, 16e6, 18e6]:
    model = get_model(f=f)
    
    print(f"a = {int(f)}, total cost:", model.objective.value())

a = 6000000, total cost: 179414495000.0
a = 8000000, total cost: 219038495000.0
a = 10000000, total cost: 258662495000.0
a = 12000000, total cost: 298286495000.0
a = 14000000, total cost: 337910495000.0
a = 16000000, total cost: 377534495000.0
a = 18000000, total cost: 417158495000.0
